In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
from transformers import AutoTokenizer
from transformers import pipeline
import torch
from transformers import AutoModelForSequenceClassification
import argparse
import pandas as pd
import numpy as np
import os
from time import time
import json
from scipy.special import softmax
import sys

/home/tajak/miniconda3/envs/emma_nova2024/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the test datasets from the GitHub repositories (access to them is obtained by request to the AGILE repository owner)

en_ginco = pd.read_json("../../datasets/EN-GINCO-test-dataset/EN-GINCO.jsonl", lines=True)
x_ginco = pd.read_json("../../datasets/X-GINCO-test-set/X-GINCO.jsonl", lines=True)

print(en_ginco.shape, x_ginco.shape)

en_ginco.head(2)

(272, 4) (790, 6)


,text,labels,dataset,language
0,Welcome to KBismarck.org! This is a community ...,Information/Explanation,EN-GINCO,English
1,Why graft thrives in postconflict zones <p> A ...,News,EN-GINCO,English


In [11]:
def predict_with_zeroshot(df_test_name):

	dfs = {
		"en-ginco": en_ginco,
		"x-ginco": x_ginco
	}

	df = dfs[df_test_name]

	texts = df["text"].to_list()

	labels = ["Other", "Information/Explanation", "News", "Instruction", "Opinion/Argumentation", "Forum", "Prose/Lyrical", "Legal", "Promotion"]

	predictions = []

	# Define the model
	classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=0)

	for i in texts:
		sequence_to_classify = i
		output = classifier(sequence_to_classify, labels, multi_label=False)

		prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(output["scores"], output["labels"])}
		predictions.append(list(prediction.keys())[0])

	# Create a json with results

	model_name = "nli"

	current_results = {
		"system": "NLI zero-shot model ([mDeBERTa-v3-base-mnli-xnli](https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-mnli-xnli))",
		"predictions": [
			{
			"train": "X-GENRE (train split)",
			"test": "{}".format(df_test_name),
			"predictions": predictions,
			}
		]
		}

	# Save the results as a new json
	with open("submissions/submission-{}-{}.json".format(model_name, df_test_name), "w") as file:
		json.dump(current_results, file)

	print("Classification with {} on {} finished.".format(model_name, df_test_name))

In [12]:
for test in ["en-ginco", "x-ginco"]:
	predict_with_zeroshot(test)

Classification with nli on en-ginco finished.
Classification with nli on x-ginco finished.
